In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import ADASYN, SMOTE

In [59]:
features = pd.read_csv('./Dyt-desktop.csv', delimiter=';').columns.values[:-1]

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
data = pd.read_csv("./Dyt-desktop.csv", delimiter=';')  # adjust path
print(data['Dyslexia'].value_counts())



# Encode categorical columns using LabelEncoder (converts to 0/1 if binary)
label_cols = ['Gender']
le = LabelEncoder()

for col in label_cols:
    data[col] = le.fit_transform(data[col])
    
data['Nativelang'] = data['Nativelang'].map({'Yes': 1, 'No': 0})
data['Otherlang'] = data['Otherlang'].map({'Yes': 1, 'No': 0})


if data['Dyslexia'].dtype == object:
    data['Dyslexia'] = le.fit_transform(data['Dyslexia'])

y=data['Dyslexia']
X=data.drop(columns=['Dyslexia'])
# Standardize numerical columns
exclude_cols = [col for col in X.columns if col.startswith('Accuracy')]
cols_to_scale = [col for col in X.columns if col not in exclude_cols]
#scaler = MinMaxScaler(feature_range=(0,1))
scaler = StandardScaler()
#X[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])
# (Optional) convert 'Dyslexia' column to 0/1 if not already


# ✅ Final dataset ready for model
#print(data.head())

print(X.head(2))
print(y.head(3))

Dyslexia
No     3252
Yes     392
Name: count, dtype: int64
   Gender  Nativelang  Otherlang  Age  Clicks1  Hits1  Misses1  Score1  \
0       1           0          1    7       10     10        0      10   
1       0           1          1   13       12     12        0      12   

   Accuracy1  Missrate1  ...  Misses31  Score31  Accuracy31  Missrate31  \
0        1.0        0.0  ...         0        0    0.000000         0.0   
1        1.0        0.0  ...         0        4    0.114286         0.0   

   Clicks32  Hits32  Misses32  Score32  Accuracy32  Missrate32  
0        17       2         0        2    0.117647    0.000000  
1        26       2         2        2    0.076923    0.076923  

[2 rows x 196 columns]
0    0
1    1
2    0
Name: Dyslexia, dtype: int32


In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42)

In [61]:
# scaler.fit(X_train[cols_to_scale])
# X_train[cols_to_scale] = scaler.transform(X_train[cols_to_scale])
# X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])

In [62]:
# from imblearn.over_sampling import BorderlineSMOTE

# sm = BorderlineSMOTE()  # or ADASYN()
# X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

from imblearn.over_sampling import ADASYN

ada = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = ada.fit_resample(X_train, y_train)

In [67]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold


skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

all_y_test = []
all_y_pred = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    model = CatBoostClassifier(
    iterations=1500,  # Number of trees
    depth=6,  # Tree depth
    learning_rate=0.03,  # Controls step size
    loss_function='Logloss',  # For binary classification
    eval_metric='F1',
    verbose=0,
    class_weights = {0: 1.0, 1:8.0 }
    #3252 / 392
    )
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

    y_pred = model.predict_proba(X_test)[:,1]
    threshold = 0.5
    y_pred_threshold = (y_pred >= threshold).astype(int)
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred_threshold)
    
print("Overall Accuracy:", accuracy_score(all_y_test, all_y_pred))
print(classification_report(all_y_test, all_y_pred))

Overall Accuracy: 0.9641975308641976
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      6510
           1       0.77      0.96      0.85       780

    accuracy                           0.96      7290
   macro avg       0.88      0.96      0.92      7290
weighted avg       0.97      0.96      0.97      7290



In [63]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


model = CatBoostClassifier(
    iterations=1500,  # Number of trees
    depth=6,  # Tree depth
    learning_rate=0.03,  # Controls step size
    loss_function='Logloss',  # For binary classification
    eval_metric='F1',
    verbose=0,
    class_weights = {0: 1.0, 1:8.0 }
    #3252 / 392
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)

y_pred = model.predict_proba(X_test)[:,1]
threshold = 0.5
y_pred_threshold = (y_pred >= threshold).astype(int)

print("Overall Accuracy:", accuracy_score(y_test, y_pred_threshold))
print(classification_report(y_test, y_pred_threshold))

Overall Accuracy: 0.821673525377229
              precision    recall  f1-score   support

           0       0.97      0.83      0.89       651
           1       0.35      0.76      0.48        78

    accuracy                           0.82       729
   macro avg       0.66      0.79      0.68       729
weighted avg       0.90      0.82      0.85       729



In [57]:
from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(
        n_estimators=100,  # You can increase this for better performance
        random_state=42
    )
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)[:,1]
threshold = 0.5
y_pred_threshold = (y_pred >= threshold).astype(int)

print("Overall Accuracy:", accuracy_score(y_test, y_pred_threshold))
print(classification_report(y_test, y_pred_threshold))

Overall Accuracy: 0.823045267489712
              precision    recall  f1-score   support

           0       0.97      0.83      0.89       651
           1       0.35      0.76      0.48        78

    accuracy                           0.82       729
   macro avg       0.66      0.79      0.69       729
weighted avg       0.90      0.82      0.85       729

